In [19]:
import pandas as pd
import glob
import numpy as np
import re

In [2]:
df1=pd.read_csv('Concat_df.csv')
df2=pd.read_csv('new_data.csv')

In [3]:
dfs = [df1, df2]

# Concaténez les DataFrames de la liste en utilisant la fonction concat()
data = pd.concat(dfs)

# Réinitialisez l'index du DataFrame concaténé
data.reset_index(drop=True, inplace=True)


In [4]:
data['Nom_Prenom']=data['Nom_Prenom'].str.upper()

In [5]:
data['Associé']=data['Associé'].fillna('Remplaçant')

In [6]:
data['Date_Heure_Fin'] = pd.to_datetime(data['Date_Heure_Fin'])
data['Date_Heure_Début'] = pd.to_datetime(data['Date_Heure_Début'])

data['Durée'] = data['Date_Heure_Fin'] - data['Date_Heure_Début']

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47510 entries, 0 to 47509
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   Source            47510 non-null  object         
 1   Nom_Prenom        47510 non-null  object         
 2   Date              47510 non-null  object         
 3   Heure_Début       47510 non-null  object         
 4   Heure_Fin         47510 non-null  object         
 5   Horaire           47510 non-null  object         
 6   Date_Heure_Fin    47510 non-null  datetime64[ns] 
 7   Date_Heure_Début  47510 non-null  datetime64[ns] 
 8   Associé           47510 non-null  object         
 9   mail              33241 non-null  object         
 10  Site              33283 non-null  object         
 11  Chef              6313 non-null   object         
 12  Formation         6269 non-null   object         
 13  Durée             47510 non-null  timedelta64[ns]
dtypes: dat

In [8]:
data.sample(3)

,Source,Nom_Prenom,Date,Heure_Début,Heure_Fin,Horaire,Date_Heure_Fin,Date_Heure_Début,Associé,mail,Site,Chef,Formation,Durée
38057,PDS,RIVIERE CONSTANCE,29/03/2023,20:00:00,08:00:00,20:00:00 - 08:00:00,2023-03-30 08:00:00,2023-03-29 20:00:00,Interne,c.riviere@imadis.fr,Marseille,NaN,NaN,0 days 12:00:00
20837,CDS,CHAILLOT PIERRE-FLEURY,21/06/2023,08:00:00,13:00:00,08:00:00 - 13:00:00,2023-06-21 13:00:00,2023-06-21 08:00:00,Associé,pf.chaillot@imadis.fr,Lyon,NaN,NaN,0 days 05:00:00
12595,RRU,NIVET HUBERT,24/04/2020,12:00:00,13:00:00,12:00:00 - 13:00:00,2020-04-24 13:00:00,2020-04-24 12:00:00,Remplaçant,NaN,NaN,NaN,NaN,0 days 01:00:00


In [9]:
data.shape

(47510, 14)

In [10]:
data.isna().sum()

Source                  0
Nom_Prenom              0
Date                    0
Heure_Début             0
Heure_Fin               0
Horaire                 0
Date_Heure_Fin          0
Date_Heure_Début        0
Associé                 0
mail                14269
Site                14227
Chef                41197
Formation           41241
Durée                   0
dtype: int64

In [11]:
data['Durée']=data['Durée'].astype(str)

In [12]:
data['Durée']=data['Durée'].str.split().str[2]

In [13]:
data['Durée'] = data['Durée'].str.replace("+ ", "")
data['Durée'] = data['Durée'].str.replace("+", "")

In [16]:
data['Durée'] = pd.to_timedelta(data['Durée'])

# Calculez la durée en heures décimales
data['Durée'] = data['Durée'].dt.total_seconds() / 3600

In [17]:
data.sample(5)

,Source,Nom_Prenom,Date,Heure_Début,Heure_Fin,Horaire,Date_Heure_Fin,Date_Heure_Début,Associé,mail,Site,Chef,Formation,Durée
39841,PDS,A POURVOIR A POURVOIR,28/05/2023,08:00:00,20:00:00,08:00:00 - 20:00:00,2023-05-28 20:00:00,2023-05-28 08:00:00,Remplaçant,NaN,Dijon,NaN,NaN,12.0
4145,PDS,THIVOLET ARNAUD,25/04/2020,18:00:00,08:00:00,18:00:00 - 08:00:00,2020-04-25 08:00:00,2020-04-25 18:00:00,Remplaçant,NaN,NaN,NaN,NaN,14.0
37980,PDS,PERREY ANTOINE,27/03/2023,18:00:00,21:00:00,18:00:00 - 21:00:00,2023-03-27 21:00:00,2023-03-27 18:00:00,Associé,a.perrey@imadis.fr,Clermont-ferrand,NaN,NaN,3.0
11569,PDS,GORINCOUR GUILLAUME,05/06/2021,08:00:00,20:00:00,08:00:00 - 20:00:00,2021-06-05 20:00:00,2021-06-05 08:00:00,Remplaçant,NaN,NaN,NaN,NaN,12.0
22015,PDS,SALA VINCENT,22/08/2021,20:00:00,08:00:00,20:00:00 - 08:00:00,2021-08-23 08:00:00,2021-08-22 20:00:00,Interne,v.sala@imadis.fr,Bordeaux,NaN,NaN,12.0


In [18]:
data.to_csv('BDD.csv', index=False)